# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 27 2022 8:30AM,249463,10,8696394-7136029,Yusen – 3D Matrix,Released
1,Oct 27 2022 8:30AM,249464,10,MSP215257,"Methapharm, Inc.",Released
2,Oct 27 2022 8:30AM,249464,10,MSP215258,"Methapharm, Inc.",Released
3,Oct 27 2022 8:30AM,249464,10,MSP215259,"Methapharm, Inc.",Released
4,Oct 27 2022 8:30AM,249464,10,MSP215260,"Methapharm, Inc.",Released
5,Oct 27 2022 8:30AM,249464,10,MSP215266,"Methapharm, Inc.",Released
6,Oct 27 2022 8:30AM,249464,10,MSP215269,"Methapharm, Inc.",Released
7,Oct 27 2022 8:30AM,249464,10,MSP215270,"Methapharm, Inc.",Released
8,Oct 27 2022 8:30AM,249464,10,MSP215274,"Methapharm, Inc.",Released
9,Oct 27 2022 8:30AM,249464,10,MSP215261,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,249460,Released,1
14,249461,Released,1
15,249462,Released,5
16,249463,Released,1
17,249464,Released,13


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249460,NaN,NaN,1.0
249461,NaN,NaN,1.0
249462,NaN,NaN,5.0
249463,NaN,NaN,1.0
249464,NaN,NaN,13.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249407,0.0,0.0,1.0
249408,0.0,0.0,1.0
249418,0.0,0.0,1.0
249447,0.0,0.0,1.0
249450,7.0,30.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249407,0,0,1
249408,0,0,1
249418,0,0,1
249447,0,0,1
249450,7,30,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249407,0,0,1
1,249408,0,0,1
2,249418,0,0,1
3,249447,0,0,1
4,249450,7,30,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249407,,,1
1,249408,,,1
2,249418,,,1
3,249447,,,1
4,249450,7,30,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 27 2022 8:30AM,249463,10,Yusen – 3D Matrix
1,Oct 27 2022 8:30AM,249464,10,"Methapharm, Inc."
14,Oct 26 2022 3:28PM,249462,10,ISDIN Corporation
19,Oct 26 2022 3:22PM,249455,10,ISDIN Corporation
35,Oct 26 2022 3:08PM,249461,22,"NBTY Global, Inc."
36,Oct 26 2022 3:07PM,249460,22,"NBTY Global, Inc."
37,Oct 26 2022 3:04PM,249459,12,"SD Head USA, LLC"
38,Oct 26 2022 2:40PM,249457,10,ISDIN Corporation
54,Oct 26 2022 2:10PM,249454,10,ISDIN Corporation
57,Oct 26 2022 1:51PM,249453,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 27 2022 8:30AM,249463,10,Yusen – 3D Matrix,,,1
1,Oct 27 2022 8:30AM,249464,10,"Methapharm, Inc.",,,13
2,Oct 26 2022 3:28PM,249462,10,ISDIN Corporation,,,5
3,Oct 26 2022 3:22PM,249455,10,ISDIN Corporation,,,16
4,Oct 26 2022 3:08PM,249461,22,"NBTY Global, Inc.",,,1
5,Oct 26 2022 3:07PM,249460,22,"NBTY Global, Inc.",,,1
6,Oct 26 2022 3:04PM,249459,12,"SD Head USA, LLC",,,1
7,Oct 26 2022 2:40PM,249457,10,ISDIN Corporation,,,16
8,Oct 26 2022 2:10PM,249454,10,ISDIN Corporation,,2,1
9,Oct 26 2022 1:51PM,249453,10,ISDIN Corporation,,,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 8:30AM,249463,10,Yusen – 3D Matrix,1,,
1,Oct 27 2022 8:30AM,249464,10,"Methapharm, Inc.",13,,
2,Oct 26 2022 3:28PM,249462,10,ISDIN Corporation,5,,
3,Oct 26 2022 3:22PM,249455,10,ISDIN Corporation,16,,
4,Oct 26 2022 3:08PM,249461,22,"NBTY Global, Inc.",1,,
5,Oct 26 2022 3:07PM,249460,22,"NBTY Global, Inc.",1,,
6,Oct 26 2022 3:04PM,249459,12,"SD Head USA, LLC",1,,
7,Oct 26 2022 2:40PM,249457,10,ISDIN Corporation,16,,
8,Oct 26 2022 2:10PM,249454,10,ISDIN Corporation,1,2,
9,Oct 26 2022 1:51PM,249453,10,ISDIN Corporation,8,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 8:30AM,249463,10,Yusen – 3D Matrix,1,,
1,Oct 27 2022 8:30AM,249464,10,"Methapharm, Inc.",13,,
2,Oct 26 2022 3:28PM,249462,10,ISDIN Corporation,5,,
3,Oct 26 2022 3:22PM,249455,10,ISDIN Corporation,16,,
4,Oct 26 2022 3:08PM,249461,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 8:30AM,249463,10,Yusen – 3D Matrix,1.0,NaN,NaN
1,Oct 27 2022 8:30AM,249464,10,"Methapharm, Inc.",13.0,NaN,NaN
2,Oct 26 2022 3:28PM,249462,10,ISDIN Corporation,5.0,NaN,NaN
3,Oct 26 2022 3:22PM,249455,10,ISDIN Corporation,16.0,NaN,NaN
4,Oct 26 2022 3:08PM,249461,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 8:30AM,249463,10,Yusen – 3D Matrix,1.0,0.0,0.0
1,Oct 27 2022 8:30AM,249464,10,"Methapharm, Inc.",13.0,0.0,0.0
2,Oct 26 2022 3:28PM,249462,10,ISDIN Corporation,5.0,0.0,0.0
3,Oct 26 2022 3:22PM,249455,10,ISDIN Corporation,16.0,0.0,0.0
4,Oct 26 2022 3:08PM,249461,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1746208,60.0,2.0,0.0
12,498877,2.0,0.0,0.0
19,498897,1.0,30.0,7.0
21,498858,2.0,0.0,0.0
22,748329,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1746208,60.0,2.0,0.0
1,12,498877,2.0,0.0,0.0
2,19,498897,1.0,30.0,7.0
3,21,498858,2.0,0.0,0.0
4,22,748329,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,60.0,2.0,0.0
1,12,2.0,0.0,0.0
2,19,1.0,30.0,7.0
3,21,2.0,0.0,0.0
4,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,60.0
1,12,Released,2.0
2,19,Released,1.0
3,21,Released,2.0
4,22,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,21,22
Status,,,,,
Completed,0.0,0.0,7.0,0.0,0.0
Executing,2.0,0.0,30.0,0.0,0.0
Released,60.0,2.0,1.0,2.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,21,22
0,Completed,0.0,0.0,7.0,0.0,0.0
1,Executing,2.0,0.0,30.0,0.0,0.0
2,Released,60.0,2.0,1.0,2.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,21,22
0,Completed,0.0,0.0,7.0,0.0,0.0
1,Executing,2.0,0.0,30.0,0.0,0.0
2,Released,60.0,2.0,1.0,2.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()